# Determinantes dos preços dos restaurantes de SP disponíveis no TripAdvisor<br>Fase 2: Extração

## 0. Pré-Scrapping
***
### Importação dos pacotes e definição de funções:

In [ ]:
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (StaleElementReferenceException, 
                                        WebDriverException, 
                                        ElementClickInterceptedException, 
                                        TimeoutException, 
                                        NoSuchElementException)

import time
from time import sleep

from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# determinando parâmetro options para a função que define o webdriver:
options = Options()
# para o browser não abrir durante o processo:
# options.add_argument("--headless")

In [1]:
# criar e atualizar variáveis html e pagina que transformam página atual do webdriver em um arquivo soup:
def atualiza_transforma_pagina(webdriver):
    global html, pagina # transforma variáveis html e pagina em globais, ou seja, não ficam restritas a função.
    html = browser.page_source
    pagina = BeautifulSoup(html, 'html.parser')
    
# clicar no botão que aceita os cookies e desobstruir página para realização da raspagem:     
def aceita_cookies(webdriver, id_):
    time.sleep(7)
    wait = WebDriverWait(webdriver, 10, ignored_exceptions=[StaleElementReferenceException, WebDriverException, NoSuchElementException])
    botao_aceita_cookies = wait.until(EC.element_to_be_clickable((By.ID, str(id_))))
    botao_aceita_cookies.click()

# clicar no botão de próxima página
def proxima_pagina(webdriver, css_locator):
    
    wait = WebDriverWait(webdriver, 10, ignored_exceptions=[StaleElementReferenceException, WebDriverException])
    global fim_da_coleta
    fim_da_coleta = 0
        
    try:
        proxima_pagina = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, str(css_locator))))
        proxima_pagina.click()
        time.sleep(5)
        
    except ElementClickInterceptedException:
        fim_da_coleta = 1

    except TimeoutException:
        fim_da_coleta = 1
        print('Erro "TimeoutException": coleta pode ter sido finalizada. Verificar!')

## 1. Scrapping
***

In [ ]:
df_restaurantes_sujo = pd.read_csv('df_restaurantes_fase1.csv')

In [ ]:
# abrindo uma lista em branco para incluir dicionários com nomes dos restaurantes e url das páginas:
cards = []

for i in range(len(df_restaurantes_sujo)):

# usar a próxima linha caso o código apresente algum erro. dessa forma, a raspagem continuará do ponto em que o ele parou:
# for i in range(número em que parou, len(df_restaurantes_sujo)):
    
    browser = webdriver.Firefox(options=options, executable_path="C:\Program Files\Mozilla Firefox\geckodriver.exe")
    url = df_restaurantes_sujo['link'][i]
    browser.get(url)

    atualiza_transforma_pagina(browser)
    aceita_cookies(browser, 'onetrust-accept-btn-handler')

    # abrindo um dicionário em branco para incluir as respectivas chaves (nomes das variáveis) e valores:
    card = {}

    if pagina.find('h1', {'class':'HjBfq'}) is not None:
        card['nome'] = pagina.find('h1', {'class':'HjBfq'}).text
    else:
        'None'   
    if df_restaurantes_sujo['link'][3000] is not None:
        card['url'] = df_restaurantes_sujo['link'][3000]
    else:
        'None'
    if pagina.find('a', {'class':'dlMOJ'}) is not None:
        card['preco'] = pagina.find('a', {'class':'dlMOJ'}).text
    else:
        'None'
    if pagina.find('a', {'href':'#MAPVIEW'}) is not None:
        card['endereco'] = pagina.find('a', {'href':'#MAPVIEW'}).text
    else:
        'None'
    if pagina.find('span', {'class':'ZDEqb'}) is not None:
        card['pontuacao'] = pagina.find('span', {'class':'ZDEqb'}).text
    else:
        'None'
    if pagina.find('a', {'href':'#REVIEWS'}) is not None:
        card['n_avaliacoes'] = pagina.find('a', {'href':'#REVIEWS'}).text
    else:
        'None'
        
    notas_chaves = []
    notas_valores = []
    
    if pagina.find_all('div', {'class':'DzMcu'}) is not None:
        notas_chaves = [i.get_text() for i in pagina.find_all('span', {'class':'BPsyj'})]
    else:
        'None'
    if pagina.find_all('div', {'class':'DzMcu'}) is not None:
        notas_valores = [i.get('class')[1] for i in pagina.find_all('span', {'class': re.compile('ui_bubble_rating')})]
    else:
        'None'
        
    notas = {}
    
    for chave, valor in zip(notas_chaves, notas_valores):
        notas[chave] = valor

    card['notas'] = notas

    detalhes_chaves = []
    detalhes_valores = []

    if pagina.find('div', {'class':'DzMcu'}) is not None: 
        detalhes_chaves = [i.get_text() for i in pagina.find_all('div', {'class':'tbUiL b'})]
    else:
        'None'
    if pagina.find('div', {'class':'SrqKb'}) is not None:
        detalhes_valores = [i.get_text() for i in pagina.find_all('div', {'class':'SrqKb'})]
    else:
        'None'

    detalhes = {}
    for chave, valor in zip(detalhes_chaves, detalhes_valores):
        detalhes[chave] = valor

    card['detalhes'] = detalhes

    cards_avaliacoes = []

    while True:
        atualiza_transforma_pagina(browser)

        avaliacoes = pagina.find_all('div', {'class': 'review-container'})

        for avaliacao in avaliacoes:
            card_avaliacao = {}

            if avaliacao.find('div', {'class':'info_text pointer_cursor'}) is not None:
                card_avaliacao['usuario'] = avaliacao.find('div', {'class':'info_text pointer_cursor'}).text
            else:
                card_avaliacao['usuario'] = 'None'
            if avaliacao.find('span', {'class':'noQuotes'}) is not None:
                card_avaliacao['titulo'] = avaliacao.find('span', {'class':'noQuotes'}).text
            else:
                card_avaliacao['titulo'] = 'None'
            if avaliacao.find('p', {'class':'partial_entry'}) is not None:
                card_avaliacao['avaliacao'] = avaliacao.find('p', {'class':'partial_entry'}).text
            else:
                card_avaliacao['avaliacao'] = 'None'
            if avaliacao.find('div', {'class':'prw_rup prw_reviews_stay_date_hsx'}) is not None:
                card_avaliacao['data_visita'] = avaliacao.find('div', {'class':'prw_rup prw_reviews_stay_date_hsx'}).text
            else:
                card_avaliacao['data_visita'] = 'None'

            cards_avaliacoes.append(card_avaliacao)

        proxima_pagina(browser, 'a.nav:nth-child(2)')

        if fim_da_coleta == 1 or len(cards_avaliacoes)>100:
            browser.quit()
            print('Coleta do restaurante, número {}, finalizada!'.format(i))
            break
        else:
            continue

    card['avaliacoes'] = cards_avaliacoes
    cards.append(card)
    
time.sleep(2)

In [ ]:
# Criando um DataFrame com os resultados
df_restaurantes_completo_sujo = pd.DataFrame(cards)
df_restaurantes_completo_sujo.to_csv('df_restaurantes_completo_sujo.csv')
df_restaurantes_completo_sujo